# Dependency resolution

In [ ]:
from start_session import start_session
root, v1, v2 = start_session()

* Most page objects in tower have a `create()` method
* .. that not only creates the top-level method
* .. but required dependencies as well

Calling `v2.job_templates.create()`, for example, creates:
* .. inventory
* .. project
* .. and job-template itself

In [ ]:
jt = v2.job_templates.create()
print(jt)

The object returned contains a dependency store (`ds`) that contains references to dependencies

In [ ]:
print(jt.ds)

Accessing dependencies is easy..

In [ ]:
print(jt.ds.inventory)

The Job Template's inventory is empty by default. Let's try creating one..

In [ ]:
jt.ds.inventory.add_host()

💡Calling `JobTemplate.create()` seems to a take a while. Why?

A: When `create()` creates a project, it also runs the project update which can take a few seconds. Project updates are one of the most time consuming operations in our tests.

### Workflow Job Templates

How would we create a workflow job template?

In [ ]:
wfjt = v2.workflow_job_templates.create()

Does WorkfowJobTemplate's `create()` method automatically create any nodes for us? 

In [ ]:
wfjt.related

In [ ]:
wfjt.related.workflow_nodes.get()

By default towerkit's dependency resolution creates the _minimum_ set of dependencies required. (This is why the inventory we saw earlier didn't have a host in it).

Let's add a node.

In [ ]:
n1 = v2.workflow_job_template_nodes.create(workflow_job_template=wfjt)

Let's see if our Workflow Job Template shows any nodes now..

In [ ]:
wfjt.related.workflow_nodes.get()

### Credentials

Let's try creating a credential..

In [ ]:
cred = v2.credentials.create()

Wait a minute.. there are lots of cred types. What kind did we just create?

In [ ]:
credential_type = cred.related.credential_type.get()
credential_type.name

Okay, but what about the other types. Can we create a cloud credential?

In [ ]:
cloud_cred = v2.credentials.create(kind='aws')
credential_type = cloud_cred.related.credential_type.get()
credential_type.name

.. net credential?

In [ ]:
net_cred = v2.credentials.create(kind='net')
credential_type = net_cred.related.credential_type.get()
credential_type.name

There's also..
* satellite
* cloudforms
* azure
* gce
* vmware
* openstack

In [ ]:
kinds = ['satellite6', 'cloudforms', 'gce', 'openstack_v3']
for kind in kinds:
    cred = v2.credentials.create(kind=kind)
    cred_type = cred.related.credential_type.get()
    print cred_type.name

What about a custom credential?

In [ ]:
custom_credential = v2.credential_types.create()
cred = v2.credentials.create(credential_type=custom_credential)

### Behind the scenes 🔍

* The `create()` method is implemented using the `has_create` mixin ([source](https://github.com/ansible/towerkit/blob/master/towerkit/api/mixins/has_create.py))
* `has_create` methods use the [toposort](https://pypi.org/project/toposort/) library to determine resource creation order ([source](https://github.com/ansible/towerkit/blob/master/towerkit/api/mixins/has_create.py#L4))
* Each page object lists its dependencies using the `optional_dependencies` class attribute ([source](https://github.com/ansible/towerkit/blob/master/towerkit/api/pages/job_templates.py#L15))

### Payloads

What if I just want to get the payload that _would_ create a resource (but not actually create the resource)?

* In addition to the `create()` method, page objects also have a `create_payload()` method that will create the payload that would be required to create the resource.

In [ ]:
v2.job_templates.create_payload()

⚠️ Note that even though `create_payload()` doesn't create the top level object (a job template in this case) it does _automatically create any dependencies_ (a project, credential, and inventory, in this case).

What if I want a payload, but don't want to automatically create any dependencies?

Use the `payload()` method.

In [ ]:
v2.job_templates.payload()